In [ ]:
import math
from textblob import TextBlob as tb
import newspaper
import os
import random
import pickle
import import_ipynb
from lexrank_summarizer import summarize
from news_classes import customizedPaper
from news_classes import paragraph

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import render_template
from flask import request

In [ ]:
cwd=os.getcwd()+'/'
news_path='news_today.pkl'

with open(news_path, 'rb') as input:
    allnews = pickle.load(input)

In [ ]:
def getSummarizedFromList(para_list):
    summ_list=[]
    for paragraph in para_list:
        if searched_word.lower() in paragraph.body.lower():
            obj={'title':paragraph.title, 'url':paragraph.url, 'text':summarize(paragraph.body, 5), 'date':paragraph.date, 'brand':paragraph.brand}
            summ_list.append(obj)    
    return summ_list

In [ ]:
def getNewsFromKey(searched_word):
    searched_word=searched_word.lower()
    para_list=[]
    for paragraph in allnews.paragraphs:
        if searched_word in paragraph.body.lower():
            if paragraph.date is not None:
                para_list.append(paragraph)
            
    para_list.sort(key=lambda item:(item.date.year, item.date.month, item.date.day), reverse=True)
    
    ret_list=[]
    for i in para_list:
        if searched_word in i.title.lower():
            ret_list.append(i)
    for i in para_list:
        if searched_word not in i.title.lower():
            ret_list.append(i)
    para_list.clear()
    return ret_list

In [ ]:
list_of_news=[]
searched_word=""
total_page=0
content_per_page=5

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/show_result', methods=['POST'])
def show_result():
    global searched_word
    searched_word = request.form['news_keyword']
    print(searched_word)
    global list_of_news
    list_of_news=getNewsFromKey(searched_word)
    global total_page
    total_page=math.ceil(len(list_of_news)/content_per_page)
    
    list_ten=[]
    for i in range(min(len(list_of_news), content_per_page)):
        list_ten.append(list_of_news[i])
    
    rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word)
    
    return rendered

In [ ]:
@app.route('/show_other_page', methods=['POST'])
def show_other_page():
    page_number = int(request.form['page_number'])
        
    list_ten=[]
    for i in range(page_number*content_per_page, min(len(list_of_news), (page_number+1)*content_per_page)):
        list_ten.append(list_of_news[i])
    
    rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word)
    
    return rendered

In [ ]:
@app.route("/")
def search_key():
    rendered = render_template('ani_search.html')
    #rendered="hello there!"
    return rendered

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('0.0.0.0', 9000, app)